<a href="https://colab.research.google.com/github/JairsonAlbertoSami/desafio-junior-data-scientist/blob/main/Analise_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Técnico - Cientista de Dados Júnior - Rio de Janeiro

Este código foi desenvolvido como parte do desafio técnico para a posição de **Cientista de Dados Júnior** no **Rio de Janeiro**. O objetivo deste desafio é demonstrar habilidades em manipulação e análise de dados python e integração com APIs.


## Integração com APIs: Feriados e Tempo

### Utilize as APIs públicas abaixo para responder às questões 1-8:
- [Public Holiday API](https://date.nager.at/Api)
- [Open-Meteo Historical Weather API](https://open-meteo.com/)


In [30]:
# Deleta as saidas da execução

from IPython.display import clear_output

In [31]:
# Instala a biblioteca requisições HTTP

!pip install requests
clear_output()

In [32]:
# importar bibliotecas necessárias
import requests
import datetime
import json

# 1.  **Quantos feriados há no Brasil em todo o ano de 2024?**

In [33]:
# Faz requisição à API e imprime o número de feriados.

response = requests.get('https://date.nager.at/api/v3/PublicHolidays/2024/BR')

if response.status_code == 200:
    holidays = response.json()
    print(f"Número total de feriados no Brasil em 2024: {len(holidays)}")
    print(" ")
    for holiday in holidays:
        print(f"{holiday['date']}: {holiday['localName']}")
else:
    print(f"Erro ao acesar a API: {response.status_code}")

Número total de feriados no Brasil em 2024: 14
 
2024-01-01: Confraternização Universal
2024-02-12: Carnaval
2024-02-13: Carnaval
2024-03-29: Sexta-feira Santa
2024-03-31: Domingo de Páscoa
2024-04-21: Dia de Tiradentes
2024-05-01: Dia do Trabalhador
2024-05-30: Corpus Christi
2024-07-09: Revolução Constitucionalista de 1932
2024-09-07: Dia da Independência
2024-10-12: Nossa Senhora Aparecida
2024-11-02: Dia de Finados
2024-11-15: Proclamação da República
2024-12-25: Natal


# 2. **Qual mês de 2024 tem o maior número de feriados?**

In [34]:
# Conta feridos por mês e identifica o mês com mais feriados, se no caso há empate de feriados mostra todos.

if response.status_code == 200:
    holidays = response.json()
    months = {}
    for holiday in holidays:
        month = holiday['date'][5:7]
        if month not in months:
            months[month] = 0
        months[month] += 1
    max_count = max(months.values())
    max_months = [month for month, count in months.items() if count == max_count]
    if len(max_months) == 1:
        print(f"O mês com mais feriados em 2024 é: {max_months[0]} ({max_count} feriados)")
    else:
        print(f"Os meses com mis feriados em 2024 são: {', '.join(max_months)} ({max_count} feriados cada)")
else:
    print(f"Erro ao acessar a API: {response.status_code}")

Os meses com mis feriados em 2024 são: 02, 03, 05, 11 (2 feriados cada)


# 3. **Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?**

In [35]:
# Conta feriados que caem em dias de uteis.

if response.status_code == 200:
    holidays = response.json()
    weekday_holidays = 0
    for holiday in holidays:
        date = holiday['date']
        day_of_week = datetime.datetime.strptime(date, '%Y-%m-%d').weekday()
        if day_of_week < 5:
            weekday_holidays += 1
    print(f"Número de feriados em dias de semana em 2024: {weekday_holidays}")
else:
    print(f"Erro ao acessar a API: {response.status_code}")


Número de feriados em dias de semana em 2024: 9


# 4. **Qual foi a temperatura média em cada mês?**
Utilize a Open-Meteo Historical Weather API para obter as temperaturas médias diárias no Rio de Janeiro de 01/01/2024 a 01/08/2024

- [Open-Meteo Historical Weather API](https://open-meteo.com/)

In [36]:
# Define as datas e a localização com o url

start_date = '2024-01-01'
end_date = '2024-08-01'
latitude = -22.9068  # Lat do RJ
longitude = -43.1729  # Long do RJ

url = (
    f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}'
    f'&longitude={longitude}&start_date={start_date}&end_date={end_date}'
    f'&daily=temperature_2m_mean&timezone=America%2FSao_Paulo')

response = requests.get(url)

In [37]:
# Obter dados históricos do clima e calculara média para cada mês.

if response.status_code == 200:
    data = response.json()
    temperatures = data['daily']['temperature_2m_mean']
    dates = data['daily']['time']

    monthly_temperatures = {}
    for date, temp in zip(dates, temperatures):
        month = date[5:7]
        if month not in monthly_temperatures:
            monthly_temperatures[month] = []
        monthly_temperatures[month].append(temp)

    for month, temps in monthly_temperatures.items():
        average_temp = sum(temps) / len(temps)
        print(f"Temperatura média em {month}/2024: {average_temp:.2f}°C")
else:
    print(f"Erro ao acessar a API: {response.status_code}")

Temperatura média em 01/2024: 26.70°C
Temperatura média em 02/2024: 27.20°C
Temperatura média em 03/2024: 26.42°C
Temperatura média em 04/2024: 25.17°C
Temperatura média em 05/2024: 24.99°C
Temperatura média em 06/2024: 22.64°C
Temperatura média em 07/2024: 21.11°C
Temperatura média em 08/2024: 21.20°C


5. # **Qual foi o tempo predominante em cada mês nesse período?**  
   Utilize como referência para o código de tempo (_weather_code_) o seguinte link: [WMO Code](https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c).


In [38]:
# Faz requisição  pra dados meteorologicos e carregar

gist_url = 'https://gist.githubusercontent.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c/raw/'

response = requests.get(gist_url)
if response.status_code == 200:
    weather_data = json.loads(response.text)
else:
    print(f"Erro ao acessar o Gist: {response.status_code}")

In [39]:
# Faz requisição  pra dados de clima diários.,

url = (f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}'
       f'&longitude={longitude}&start_date={start_date}&end_date={end_date}'
       f'&daily=weathercode&timezone=America%2FSao_Paulo')

response = requests.get(url)

In [40]:
if response.status_code == 200:
    data = response.json()
    weather_codes = data['daily']['weathercode']
    dates = data['daily']['time']

    # Agrupa os códigos de clima por mês

    monthly_weather_codes = {}
    for date, code in zip(dates, weather_codes):
        month = date[5:7]
        if month not in monthly_weather_codes:
            monthly_weather_codes[month] = []
        monthly_weather_codes[month].append(code)

    # Determina o código de clima predominan por mês e imprime a descrição através do WMO Code.
    for month, codes in monthly_weather_codes.items():
        predominant_code = max(set(codes), key=codes.count)
        if str(predominant_code) in weather_data:
            description = weather_data[str(predominant_code)]['day']['description']
        else:
            description = "Código desconhecido"

        print(f"Tempo predominante em {month}/2024: {description}, com código WMO Code {predominant_code}")
else:
    print(f"Erro ao acessar a API: {response.status_code}")


Tempo predominante em 01/2024: Rain, com código WMO Code 63
Tempo predominante em 02/2024: Light Drizzle, com código WMO Code 51
Tempo predominante em 03/2024: Light Drizzle, com código WMO Code 51
Tempo predominante em 04/2024: Cloudy, com código WMO Code 3
Tempo predominante em 05/2024: Light Drizzle, com código WMO Code 51
Tempo predominante em 06/2024: Mainly Sunny, com código WMO Code 1
Tempo predominante em 07/2024: Cloudy, com código WMO Code 3
Tempo predominante em 08/2024: Partly Cloudy, com código WMO Code 2


# 6. **Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?**


In [41]:
# # Faz uma requisição para obter a temperatura média e códigos de clima diários.

url_weather = (f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}'
              f'&longitude={longitude}&start_date={start_date}&end_date={end_date}'
              f'&daily=temperature_2m_mean,weathercode&timezone=America%2FSao_Paulo')

response_weather = requests.get(url_weather)

In [42]:
# Verifica se a requisição, contrário inicializar litas vazias.

if response_weather.status_code == 200:
    data_weather = response_weather.json()
    temperatures = data_weather['daily']['temperature_2m_mean']
    weather_codes = data_weather['daily']['weathercode']
    dates = data_weather['daily']['time']
else:
    print(f"Erro ao acessar a API de clima: {response_weather.status_code}")
    temperatures = []
    weather_codes = []
    dates = []

In [43]:
# Itera sobre feriados  e encontra a temperatura e o clima correspondente
#
for holiday in holidays:
    holiday_date = holiday['date']
    if holiday_date in dates:
        index = dates.index(holiday_date)
        temperature = temperatures[index]
        weather_code = weather_codes[index]

        if str(weather_code) in weather_data:
            weather_description = weather_data[str(weather_code)]['day']['description']
        else:
            weather_description = "Código desconhecido"
        print(f"{holiday_date} ({holiday['localName']}): Temperatura média: {temperature:.2f}°C, Tempo: {weather_description}")


2024-01-01 (Confraternização Universal): Temperatura média: 24.90°C, Tempo: Light Drizzle
2024-02-12 (Carnaval): Temperatura média: 30.20°C, Tempo: Mainly Sunny
2024-02-13 (Carnaval): Temperatura média: 30.50°C, Tempo: Light Drizzle
2024-03-29 (Sexta-feira Santa): Temperatura média: 25.00°C, Tempo: Light Rain
2024-03-31 (Domingo de Páscoa): Temperatura média: 24.50°C, Tempo: Drizzle
2024-04-21 (Dia de Tiradentes): Temperatura média: 23.10°C, Tempo: Partly Cloudy
2024-05-01 (Dia do Trabalhador): Temperatura média: 28.10°C, Tempo: Sunny
2024-05-30 (Corpus Christi): Temperatura média: 20.80°C, Tempo: Light Drizzle
2024-07-09 (Revolução Constitucionalista de 1932): Temperatura média: 21.50°C, Tempo: Drizzle


7. **Considere as seguintes suposições:**
   - O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
   - Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
   - O cidadão carioca só vai à praia quando não está com frio;
   - O cidadão carioca também só vai à praia em dias com sol, evitando dias **totalmente** nublados ou chuvosos (considere _weather_code_ para determinar as condições climáticas).

   Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

# ***Resposta: 7***

Em 2024, considerando a cultura carioca e as condições climáticas, alguns feriados foram considerados "não aproveitáveis" para ir à praia.

Os cariocas tendem a aproveitar feriados longos, especialmente se caírem em uma sexta-feira ou segunda-feire, dito feriadão, pois permitem mais tempo para lazer e descanso. No entanto, a possibilidade de ir à praia é limitada por fatores como temperatura e condições climáticas.

Feriados como o Confraternização Universal (01/01) e Corpus Christi (30/05) apresentaram condições de Light Drizzle e temperaturas relativamente baixas, o que não favoreceu a ida à praia. O Carnaval (13/02) e o Domingo de Páscoa (31/03) também tiveram condições de
 Drizzle ou Light Drizzle, tornando-os menos propícios para aproveitar a praia. Feriados como a Sexta-feira Santa (29/03) e Dia de Tiradentes (21/04) tiveram temperaturas agradáveis, mas o tempo Light Rain ou Partly Cloudy limitou as atividades ao ar livre.

Portanto, apesar de alguns feriados coincidirem com finais de semana prolongados, as condições climáticas desfavoráveis fizeram com que a maioria dos feriados não fosse ideal para a tradicional ida à praia carioca.

# 8. **Qual foi o feriado "mais aproveitável" de 2024?**  
   Considere o melhor par tempo e temperatura.

# ***Resposta: 8***

O Carnaval (12/02 e 13/02) é um evento culturalmente significativo no Rio de Janeiro. No dia 12, a temperatura foi bastante agradável (30.20°C) e o tempo foi predominantemente Mainly Sunny. No dia 13, que também é feriado de Carnaval, a temperatura foi igualmente alta (30.50°C), mas com Light Drizzle. Apesar das condições climáticas não serem ideais para a praia no segundo dia, a celebração e a importância do Carnaval fazem com que esse feriado seja muito valorizado pelos cariocas.

Portanto, mesmo que as condições climáticas do dia 13 não fossem perfeitas para a praia, a relevância cultural e a celebração do Carnaval fazem com que o feriado seja considerado altamente aproveitável para a maioria dos cariocas. Assim, podemos concluir que o Carnaval (12/02 e 13/02) é o feriado mais aproveitável de 2024, levando em conta a importância cultural e a combinação de temperatura alta e predominância de Mainly Sunny no primeiro dia.

No entanto, se considerarmos apenas as condições climáticas ideais para um dia de praia, o Dia do Trabalhador (01/05), com uma temperatura média de 28.10°C e tempo ensolarado (Sunny), oferece as melhores condições para aproveitar a praia no Rio de Janeiro.





